In [1]:
from __future__ import annotations

import os
from typing import List

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
import numpy as np
import uproot

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [2]:
MAIN_DIR = "../../../../"

plot_dir = f"{MAIN_DIR}/plots/GoF/23May13"
_ = os.system(f"mkdir -p {plot_dir}")

In [3]:
cards_dir = "23May13MP"

file = uproot.concatenate(
    f"/uscms/home/rkansal/nobackup/CMSSW_11_2_0/src/cards/{cards_dir}/higgsCombineData.GoodnessOfFit.mH125.root"
)

data_ts = file["limit"][0]  # data test statistic

file = uproot.concatenate(
    f"/uscms/home/rkansal/nobackup/CMSSW_11_2_0/src/cards/{cards_dir}/higgsCombineToys.GoodnessOfFit.mH125.*.root"
)

toy_ts = np.array(file["limit"])  # toy test statistics

In [ ]:
def p_value(data_ts: float, toy_ts: List[float]):
    return np.mean(toy_ts >= data_ts)

In [ ]:
pval = p_value(data_ts, toy_ts)
pval

In [ ]:
plot_max = max(np.max(toy_ts), data_ts)
plot_min = min(np.min(toy_ts), data_ts)

In [ ]:
plt.figure(figsize=(12, 8))
_ = plt.hist(
    toy_ts, np.linspace(plot_min, plot_max, 10), color="dimgrey", histtype="step", label="Toys"
)
plt.axvline(data_ts, color="red", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")
_ = plt.legend()
plt.title("Goodness of Fit")
plt.ylabel("Number of Toys")
plt.xlabel("Test Statistics")
plt.savefig(f"{plot_dir}/gof.pdf")

In [ ]:
from scipy.stats import chi2

chi2_df = 100

plt.figure(figsize=(12, 8))
x = np.linspace(plot_min, plot_max, 101)
plt.plot(
    x,
    chi2.pdf(x, chi2_df) * 1.3,
    color="blue",
    linestyle="--",
    alpha=0.6,
    label=rf"$\chi^2_{{DoF = {chi2_df}}}$",
)
_ = plt.hist(
    toy_ts,
    np.linspace(plot_min, plot_max, 10),
    density=True,
    color="dimgrey",
    histtype="step",
    label="Toys",
)
plt.axvline(data_ts, color="red", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")
_ = plt.legend()
plt.title("Goodness of Fit")
plt.ylabel("Number of Toys (A.U.)")
plt.xlabel("Test Statistics")
plt.savefig(f"{plot_dir}/gof_chi2.pdf")